# **MLProcess - Air Quality**
---
**4 - Modeling, Tuning, and Evaluation**

In [1]:
# Import the required libraries.
import copy
import hashlib
import json

# Need to be installed.
import joblib
import time
import yaml
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression as LGR
from sklearn.ensemble import BaggingClassifier as BGC, RandomForestClassifier as RFC, AdaBoostClassifier as ABC, GradientBoostingClassifier as GBC

from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from datetime import datetime
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

## **0 - Configuration File**
---

In [2]:
# Function to load configuration parameter.
def load_config(path_config):
    """
    Load the configuration file (config.yaml).

    Parameters:
    ----------
    path_config : str
        Configuration file location.

    Returns:
    -------
    params : dict
        The configuration parameters.
    """

    # Try to load config.yaml file.
    try:
        with open(path_config, 'r') as file:
            params = yaml.safe_load(file)
    except FileNotFoundError as err:
        raise RuntimeError(f"Configuration file not found in {path_config}")

    return params

In [3]:
# Function to update configuration parameter.
def update_config(key, value, params, path_config):
    """
    Update the configuration parameter values.

    Parameters:
    ----------
    key : str
        The key to be updated.

    value : any type supported in Python
        The updated value.

    params : dict
        Loaded configuration parameters.

    path_config : str
        Configuration file location.

    Returns:
    -------
    config : dict
        Updated configuration parameters.
    """

    # To maintain the raw config immutable.
    params = params.copy()

    # Update the configuration parameters.
    params[key] = value

    with open(path_config, 'w') as file:
        yaml.dump(params, file)

    print(f"Params Updated! \nKey: {key} \nValue: {value}\n")

    # Reload the updated configuration parameters.
    config = load_config(path_config)

    return config

In [4]:
# Load the configuration file.
PATH_CONFIG = "../config/config.yaml"
config = load_config(PATH_CONFIG)

In [5]:
# Check the configuration parameters.
config

{'datetime_columns': ['tanggal'],
 'features': ['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2'],
 'impute_co': 11.0,
 'impute_no2': 18.0,
 'impute_o3': 29.0,
 'impute_pm10': {'BAIK': 28.944444444444443, 'TIDAK BAIK': 55.61550632911393},
 'impute_pm25': {'BAIK': 39.559633027522935, 'TIDAK BAIK': 82.7256289308176},
 'impute_so2': 35.042940320232894,
 'int32_columns': ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max'],
 'label': 'category',
 'label_categories': ['BAIK', 'SEDANG', 'TIDAK SEHAT'],
 'label_categories_new': ['BAIK', 'TIDAK BAIK'],
 'object_columns': ['stasiun', 'critical', 'category'],
 'path_joined_data': '../data/interim/joined_dataset.pkl',
 'path_ohe_stasiun': '../models/ohe_stasiun.pkl',
 'path_raw_data': '../data/raw/',
 'path_test_set': ['../data/interim/X_test.pkl', '../data/interim/y_test.pkl'],
 'path_train_set': ['../data/interim/X_train.pkl',
  '../data/interim/y_train.pkl'],
 'path_valid_set': ['../data/interim/X_valid.pkl',
  '../data/interim/y_valid.pkl'],


## **1 - Load Data**
---

In [6]:
# Undersampling
X_rus = joblib.load(f"../data/processed/X_rus.pkl")
y_rus = joblib.load(f"../data/processed/y_rus.pkl")

# Oversampling
X_ros = joblib.load(f"../data/processed/X_ros.pkl")
y_ros = joblib.load(f"../data/processed/y_ros.pkl")

# Smote
X_sm = joblib.load(f"../data/processed/X_sm.pkl")
y_sm = joblib.load(f"../data/processed/y_sm.pkl")

# Valid
X_valid = joblib.load('../data/processed/X_valid_feng.pkl')
y_valid = joblib.load('../data/processed/y_valid_feng.pkl')

# Test
X_test = joblib.load('../data/processed/X_test_feng.pkl')
y_test = joblib.load('../data/processed/y_test_feng.pkl')

## **2 - Training Log**
---

In [7]:
# Function to support training log.
def time_stamp():
    return datetime.now()

def create_log_template():
    logger = {
        'model_name' : [],
        'model_id' : [],
        'training_time' : [],
        'training_date' : [],
        'performance' : [],
        'f1_score_avg' : [],
        'data_configuration' : []
    }
    
    return logger

In [8]:
def update_training_log(current_log, path_log):
    """
    Update the training log.

    Parameters:
    ----------
    current_log : dict
        The training log current state.

    path_log : str
        The directory of training log.

    Returns:
    -------
    last_log : dict
        The updated training log.
    """

    # Ensure the current log immutable.
    current_log = current_log.copy()

    # Open the training log file.
    try:
        with open(path_log, 'r') as file:
            last_log = json.load(file)
    # If the training log does not exists.
    except FileNotFoundError as err:
        # Create the new training log.
        with open(path_log, 'w') as file:
            file.write("[]")

        # Reload the current training log.
        with open(path_log, 'r') as file:
            last_log = json.load(file)

    last_log.append(current_log)

    # Rewrite the training log with the updated one.
    with open(path_log, 'w') as file:
        json.dump(last_log, file)

    return last_log

## **3 - Model Training**
---

In [9]:
# Create model object.
baseline = DummyClassifier()
knn = KNN()
dtc = DTC()
lgr = LGR()
bgc = BGC()
rfc = RFC()
abc = ABC()
gbc = GBC()

In [10]:
list_of_model = {
    "undersampling": [
        {"model_name": baseline.__class__.__name__, "model_object": baseline, "model_id": ""},
        {"model_name": knn.__class__.__name__, "model_object": knn, "model_id": ""},
        {"model_name": dtc.__class__.__name__, "model_object": dtc, "model_id": ""},
        {"model_name": lgr.__class__.__name__, "model_object": lgr, "model_id": ""},
        {"model_name": bgc.__class__.__name__, "model_object": bgc, "model_id": ""},
        {"model_name": rfc.__class__.__name__, "model_object": rfc, "model_id": ""},
        {"model_name": abc.__class__.__name__, "model_object": abc, "model_id": ""},
        {"model_name": gbc.__class__.__name__, "model_object": gbc, "model_id": ""}
    ],
    "oversampling": [
        {"model_name": baseline.__class__.__name__, "model_object": baseline, "model_id": ""},
        {"model_name": knn.__class__.__name__, "model_object": knn, "model_id": ""},
        {"model_name": dtc.__class__.__name__, "model_object": dtc, "model_id": ""},
        {"model_name": lgr.__class__.__name__, "model_object": lgr, "model_id": ""},
        {"model_name": bgc.__class__.__name__, "model_object": bgc, "model_id": ""},
        {"model_name": rfc.__class__.__name__, "model_object": rfc, "model_id": ""},
        {"model_name": abc.__class__.__name__, "model_object": abc, "model_id": ""},
        {"model_name": gbc.__class__.__name__, "model_object": gbc, "model_id": ""}
    ],
    "smote": [
        {"model_name": baseline.__class__.__name__, "model_object": baseline, "model_id": ""},
        {"model_name": knn.__class__.__name__, "model_object": knn, "model_id": ""},
        {"model_name": dtc.__class__.__name__, "model_object": dtc, "model_id": ""},
        {"model_name": lgr.__class__.__name__, "model_object": lgr, "model_id": ""},
        {"model_name": bgc.__class__.__name__, "model_object": bgc, "model_id": ""},
        {"model_name": rfc.__class__.__name__, "model_object": rfc, "model_id": ""},
        {"model_name": abc.__class__.__name__, "model_object": abc, "model_id": ""},
        {"model_name": gbc.__class__.__name__, "model_object": gbc, "model_id": ""}
    ]
}

In [11]:
def train_eval_model(list_of_model, prefix_model_name, X_train, y_train, data_configuration, X_valid, y_valid, path_log):
    list_of_model = copy.deepcopy(list_of_model)
    
    logger = create_log_template()
    
    for model in tqdm(list_of_model):
        model_name = f"{prefix_model_name} - {model['model_name']}"
        
        start_time = time_stamp()
        model['model_object'].fit(X_train, y_train)
        finished_time = time_stamp()
        
        elapsed_time = finished_time - start_time
        elapsed_time = elapsed_time.total_seconds()
        
        y_pred = model['model_object'].predict(X_valid)
        performance = classification_report(y_valid, y_pred, output_dict=True)
        
        plain_id = str(start_time) + str(finished_time)
        cipher_id = hashlib.md5(plain_id.encode()).hexdigest()
        
        model['model_id'] = cipher_id
        
        logger['model_name'].append(model_name)
        logger['model_id'].append(cipher_id)
        logger['training_time'].append(elapsed_time)
        logger['training_date'].append(str(start_time))
        logger['performance'].append(performance)
        logger['f1_score_avg'].append(performance['macro avg']['f1-score'])
        logger['data_configuration'].append(data_configuration)
        
    training_log = update_training_log(logger, path_log)        
    
    return training_log, list_of_model

In [12]:
PATH_LOG = "../logs/training_log.json"

training_log, list_of_model_rus = train_eval_model(
    list_of_model = list_of_model['undersampling'],
    prefix_model_name = "first",
    X_train = X_rus,
    y_train = y_rus,
    data_configuration = "undersampling",
    X_valid = X_valid,
    y_valid = y_valid,
    path_log = PATH_LOG
)

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 27.89it/s]


In [13]:
list_of_model['undersampling'] = copy.deepcopy(list_of_model_rus)

In [14]:
training_log, list_of_model_ros = train_eval_model(
    list_of_model['oversampling'],
    "first",
    X_ros,
    y_ros,
    "oversampling",
    X_valid,
    y_valid,
    PATH_LOG
)

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 21.98it/s]


In [15]:
list_of_model['oversampling'] = copy.deepcopy(list_of_model_ros)

In [16]:
training_log, list_of_model_sm = train_eval_model(
    list_of_model['smote'],
    "first",
    X_sm,
    y_sm,
    "smote",
    X_valid,
    y_valid,
    PATH_LOG
)

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 15.74it/s]


In [17]:
list_of_model['smote'] = copy.deepcopy(list_of_model_sm)

In [18]:
def training_log_to_df(training_log):
    training_res = pd.DataFrame()
    
    for log in tqdm(training_log):
        training_res = pd.concat([training_res, pd.DataFrame(log)])
        
    training_res.sort_values(['f1_score_avg', 'training_time'],
                             ascending = [False, True],
                             inplace = True)
    
    training_res.reset_index(inplace=True, drop=True)
    
    return training_res

In [19]:
training_res = training_log_to_df(training_log)

100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 424.80it/s]


In [20]:
training_res

,model_name,model_id,training_time,training_date,performance,f1_score_avg,data_configuration
0,first - DecisionTreeClassifier,05d0c752da777fbdf763c8e3925e1052,0.002106,2026-02-12 09:25:03.152635,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,oversampling
1,first - DecisionTreeClassifier,7a3100a0d25a7acb7ede3d7c8cfd2855,0.006668,2026-02-12 09:25:05.150128,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,smote
2,first - BaggingClassifier,60b243a3e4c63d4afd596b195ad63d3b,0.017581,2026-02-12 09:25:03.164503,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,oversampling
3,first - BaggingClassifier,7d0a6214acac89e889e3d54f7e3cc4e4,0.032121,2026-02-12 09:25:05.173719,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,smote
4,first - AdaBoostClassifier,2b8e78353d998d5363c2cd799467f2a6,0.055279,2026-02-12 09:25:03.287959,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,oversampling
5,first - AdaBoostClassifier,42d79aba52f06035062cb79059aa9eec,0.069438,2026-02-12 09:25:05.331492,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,smote
6,first - RandomForestClassifier,b2f3aae7ec94edb0941e2d74d7a84682,0.095607,2026-02-12 09:25:03.185876,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,oversampling
7,first - RandomForestClassifier,be6844bee1ec682b7947145a33d47b9d,0.115634,2026-02-12 09:25:05.209484,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,smote
8,first - GradientBoostingClassifier,59c837777c0479edf1f05def6168ee0b,0.141363,2026-02-12 09:25:03.347734,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,oversampling
9,first - GradientBoostingClassifier,2b27c426be85b7ff7df945ff02c76963,0.220088,2026-02-12 09:25:05.405338,"{'0': {'precision': 1.0, 'recall': 1.0, 'f1-sc...",1.000000,smote


## **4 - Model Tuning**
---

In [21]:
# Function to evaluate model.
def evaluate_model(estimator, param_space, X_train, y_train):
    """
    Do hyperparameter tuning and cross validation.
    """
    
    # Create tuner object.
    tuner = RandomizedSearchCV(
        estimator = estimator,
        param_distributions = param_space,
        n_iter = 100,
        # param_grid = param_space,
        scoring = "f1",
        cv = 5,
        return_train_score = True,
        n_jobs = -1,
        verbose = 1
    )

    # Fit the tuner -> CV + HT.
    tuner.fit(X_train, y_train)

    # Get the best model.
    best_model = tuner.best_estimator_

    # Predict the data.
    pred = best_model.predict(X_train)

    # Get the scores from the best model.
    best_index = tuner.best_index_
    train_f1 = tuner.cv_results_["mean_train_score"][best_index]
    cv_f1 = tuner.cv_results_["mean_test_score"][best_index]

    print(f"Train F1 : {train_f1}")
    print(f"CV F1    : {cv_f1}")

    return best_model, train_f1, cv_f1

In [22]:
# Hyperparameter for KNN.
knn_params = {
    "n_neighbors": np.arange(1, 13),
    "weights": ["uniform", "distance"],
    "p": [1, 2]
}

# Hyperparameter for LGR.
C = [0.01, 0.1, 1.0, 10.0]

# Hyperparameter for DTC, RFC, and GBC.
DEPTH = np.arange(1, 13)

# Hyperparameter for BGC, RFC, ABC, and GBC.
B = [10, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

# Hyperparameter for ABC and GBC.
LR = [0.001, 0.01, 0.05, 0.1, 1]

Assume that we tune our model on smote data.

### KNN

In [23]:
# Record the start time.
start_time = time.time()

# Evaluate the model.
knn_best_sm, knn_f1_sm, knn_f1_cv_sm = evaluate_model(
    estimator = knn,
    param_space = knn_params,
    X_train = X_sm,
    y_train = y_sm
)

# Record the end time.
end_time = time.time()

# Calculate the total fitting time.
fitting_time = end_time - start_time
minutes, seconds = divmod(fitting_time, 60)
print(f"Fitting Time : {int(minutes):02d} minutes {int(seconds):02d} seconds\n")

knn_best_sm

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Train F1 : 1.0
CV F1    : 0.9863069805536149
Fitting Time : 00 minutes 03 seconds



,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",np.int64(1)
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",1
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


### LGR

In [24]:
# Define hyperparameter space.
lgr_params = {
    "C": C
}

# Record the start time.
start_time = time.time()

# Evaluate the model.
lgr_best_sm, lgr_f1_sm, lgr_f1_cv_sm = evaluate_model(
    estimator = lgr,
    param_space = lgr_params,
    X_train = X_sm,
    y_train = y_sm
)

# Record the end time.
end_time = time.time()

# Calculate the total fitting time.
fitting_time = end_time - start_time
minutes, seconds = divmod(fitting_time, 60)
print(f"Fitting Time : {int(minutes):02d} minutes {int(seconds):02d} seconds\n")

lgr_best_sm

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Train F1 : 0.9767284331058846
CV F1    : 0.9772788273265155
Fitting Time : 00 minutes 00 seconds



,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",10.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mu

### DTC

In [25]:
# Define hyperparameter space.
dtc_params = {
    "max_depth": DEPTH
}

# Record the start time.
start_time = time.time()

# Evaluate the model.
dtc_best_sm, dtc_f1_sm, dtc_f1_cv_sm = evaluate_model(
    estimator = dtc,
    param_space = dtc_params,
    X_train = X_sm,
    y_train = y_sm
)

# Record the end time.
end_time = time.time()

# Calculate the total fitting time.
fitting_time = end_time - start_time
minutes, seconds = divmod(fitting_time, 60)
print(f"Fitting Time : {int(minutes):02d} minutes {int(seconds):02d} seconds\n")

dtc_best_sm

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Train F1 : 1.0
CV F1    : 0.99769521236174
Fitting Time : 00 minutes 00 seconds



,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",np.int64(4)
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at th

### BGC

In [26]:
# Define hyperparameter space.
bgc_params = {
    "n_estimators": B
}

# Record the start time.
start_time = time.time()

# Evaluate the model.
bgc_best_sm, bgc_f1_sm, bgc_f1_cv_sm = evaluate_model(
    estimator = bgc,
    param_space = bgc_params,
    X_train = X_sm,
    y_train = y_sm
)


# Record the end time.
end_time = time.time()

# Calculate the total fitting time.
fitting_time = end_time - start_time
minutes, seconds = divmod(fitting_time, 60)
print(f"Fitting Time : {int(minutes):02d} minutes {int(seconds):02d} seconds\n")

bgc_best_sm

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Train F1 : 1.0
CV F1    : 0.9988424398952007
Fitting Time : 00 minutes 05 seconds



,"estimator estimator: object, default=NoneThe base estimator to fit on random subsets of the dataset.If None, then the base estimator is a:class:`~sklearn.tree.DecisionTreeClassifier`... versionadded:: 1.2 `base_estimator` was renamed to `estimator`.",None
,"n_estimators n_estimators: int, default=10The number of base estimators in the ensemble.",50
,"max_samples max_samples: int or float, default=NoneThe number of samples to draw from X to train each base estimator (withreplacement by default, see `bootstrap` for more details).- If None, then draw `X.shape[0]` samples irrespective of `sample_weight`.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` unweighted samples or `max_samples * sample_weight.sum()` weighted samples.",None
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator (without replacement by default, see `bootstrap_features` for moredetails).- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.",1.0
,"bootstrap bootstrap: bool, default=TrueWhether samples are drawn with replacement. If False, sampling withoutreplacement is performed. If fitting with `sample_weight`, it isstrongly recommended to choose True, as only drawing with replacementwill ensure the expected frequency semantics of `sample_weight`.",True
,"bootstrap_features bootstrap_features: bool, default=FalseWhether features are drawn with replacement.",False
,"oob_score oob_score: bool, default=FalseWhether to use out-of-bag samples to estimatethe generalization error. Only available if bootstrap=True.",False
,"warm_start warm_start: bool, default=FalseWhen set to True, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fita whole new ensemble. See :term:`the Glossary `... versionadded:: 0.17 *warm_start* constructor parameter.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for both :meth:`fit` and:meth:`predict`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the random resampling of the original dataset(sample wise and feature wise).If the base estimator accepts a `random_state` attribute, a differentseed is generated for each instance in the ensemble.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",None
,"verbose verbose: int, default=0Controls the verbosity when fitting and predicting.",0


### RFC

In [27]:
# Define hyperparameter space.
rfc_params = {
    "n_estimators": B,
    "max_depth": DEPTH
}

# Record the start time.
start_time = time.time()

# Evaluate the model.
rfc_best_sm, rfc_f1_sm, rfc_f1_cv_sm = evaluate_model(
    estimator = rfc,
    param_space = rfc_params,
    X_train = X_sm,
    y_train = y_sm
)

# Record the end time.
end_time = time.time()

# Calculate the total fitting time.
fitting_time = end_time - start_time
minutes, seconds = divmod(fitting_time, 60)
print(f"Fitting Time : {int(minutes):02d} minutes {int(seconds):02d} seconds\n")

rfc_best_sm

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Train F1 : 1.0
CV F1    : 0.9988439306358382
Fitting Time : 00 minutes 29 seconds



,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",np.int64(7)
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature 

### ABC

In [28]:
# Define hyperparameter space.
abc_params = {
    "n_estimators": B,
    "learning_rate": LR
}

# Record the start time.
start_time = time.time()

# Evaluate the model.
abc_best_sm, abc_f1_sm, abc_f1_cv_sm = evaluate_model(
    estimator = abc,
    param_space = abc_params,
    X_train = X_sm,
    y_train = y_sm
)

# Record the end time.
end_time = time.time()

# Calculate the total fitting time.
fitting_time = end_time - start_time
minutes, seconds = divmod(fitting_time, 60)
print(f"Fitting Time : {int(minutes):02d} minutes {int(seconds):02d} seconds\n")

abc_best_sm

Fitting 5 folds for each of 55 candidates, totalling 275 fits
Train F1 : 1.0
CV F1    : 0.9984732824427482
Fitting Time : 00 minutes 21 seconds



,"estimator estimator: object, default=NoneThe base estimator from which the boosted ensemble is built.Support for sample weighting is required, as well as proper``classes_`` and ``n_classes_`` attributes. If ``None``, thenthe base estimator is :class:`~sklearn.tree.DecisionTreeClassifier`initialized with `max_depth=1`... versionadded:: 1.2 `base_estimator` was renamed to `estimator`.",None
,"n_estimators n_estimators: int, default=50The maximum number of estimators at which boosting is terminated.In case of perfect fit, the learning procedure is stopped early.Values must be in the range `[1, inf)`.",300
,"learning_rate learning_rate: float, default=1.0Weight applied to each classifier at each boosting iteration. A higherlearning rate increases the contribution of each classifier. There isa trade-off between the `learning_rate` and `n_estimators` parameters.Values must be in the range `(0.0, inf)`.",1
,"random_state random_state: int, RandomState instance or None, default=NoneControls the random seed given at each `estimator` at eachboosting iteration.Thus, it is only used when `estimator` exposes a `random_state`.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",None


### GBC

In [29]:
# Define hyperparameter space.
gbc_params = {
    "n_estimators": B,
    "learning_rate": LR,
    "max_depth": DEPTH
}

# Record the start time.
start_time = time.time()

# Evaluate the model.
gbc_best_sm, gbc_f1_sm, gbc_f1_cv_sm = evaluate_model(
    estimator = gbc,
    param_space = gbc_params,
    X_train = X_sm,
    y_train = y_sm
)

# Record the end time.
end_time = time.time()

# Calculate the total fitting time.
fitting_time = end_time - start_time
minutes, seconds = divmod(fitting_time, 60)
print(f"Fitting Time : {int(minutes):02d} minutes {int(seconds):02d} seconds\n")

gbc_best_sm

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Train F1 : 0.9992295653039609
CV F1    : 0.9984570834404802
Fitting Time : 00 minutes 42 seconds



,"loss loss: {'log_loss', 'exponential'}, default='log_loss'The loss function to be optimized. 'log_loss' refers to binomial andmultinomial deviance, the same as used in logistic regression.It is a good choice for classification with probabilistic outputs.For loss 'exponential', gradient boosting recovers the AdaBoost algorithm.",'log_loss'
,"learning_rate learning_rate: float, default=0.1Learning rate shrinks the contribution of each tree by `learning_rate`.There is a trade-off between learning_rate and n_estimators.Values must be in the range `[0.0, inf)`.For an example of the effects of this parameter and its interaction with``subsample``, see:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_regularization.py`.",0.01
,"n_estimators n_estimators: int, default=100The number of boosting stages to perform. Gradient boostingis fairly robust to over-fitting so a large number usuallyresults in better performance.Values must be in the range `[1, inf)`.",400
,"subsample subsample: float, default=1.0The fraction of samples to be used for fitting the individual baselearners. If smaller than 1.0 this results in Stochastic GradientBoosting. `subsample` interacts with the parameter `n_estimators`.Choosing `subsample < 1.0` leads to a reduction of varianceand an increase in bias.Values must be in the range `(0.0, 1.0]`.",1.0
,"criterion criterion: {'friedman_mse', 'squared_error'}, default='friedman_mse'The function to measure the quality of a split. Supported criteria are'friedman_mse' for the mean squared error with improvement score byFriedman, 'squared_error' for mean squared error. The default value of'friedman_mse' is generally the best as it can provide a betterapproximation in some cases... versionadded:: 0.18",'friedman_mse'
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, values must be in the range `[2, inf)`.- If float, values must be in the range `(0.0, 1.0]` and `min_samples_split` will be `ceil(min_samples_split * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, values must be in the range `[1, inf)`.- If float, values must be in the range `(0.0, 1.0)` and `min_samples_leaf` will be `ceil(min_samples_leaf * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.Values must be in the range `[0.0, 0.5]`.",0.0
,"max_depth max_depth: int or None, default=3Maximum depth of the individual regression estimators. The maximumdepth limits the number of nodes in the tree. Tune this parameterfor best performance; the best value depends on the interactionof the input variables. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.If int, values must be in the range `[1, inf)`.",np.int64(2)
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.Values must be in the range `[0.0, inf)`.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``

## **4 - Performance Summary**
---

In [30]:
# Create a summary dataframe of train vs CV performance.
performance = {
    "Model": ["KNN", "LGR", "DTC", "BGC", "RFC", "ABC", "GBC"],
    "Train F1": [knn_f1_sm, lgr_f1_sm, dtc_f1_sm, bgc_f1_sm, rfc_f1_sm, abc_f1_sm, gbc_f1_sm],
    "CV F1": [knn_f1_cv_sm, lgr_f1_cv_sm, dtc_f1_cv_sm, bgc_f1_cv_sm, rfc_f1_cv_sm, abc_f1_cv_sm, gbc_f1_cv_sm]
}

summary_df = pd.DataFrame(performance)
summary_df

,Model,Train F1,CV F1
0,KNN,1.000000,0.986307
1,LGR,0.976728,0.977279
2,DTC,1.000000,0.997695
3,BGC,1.000000,0.998842
4,RFC,1.000000,0.998844
5,ABC,1.000000,0.998473
6,GBC,0.999230,0.998457


## **5 - Model Serialization**
---

In [31]:
# Model Serialization.
PATH_PRODUCTION_MODEL = "../models/best_model.pkl"
joblib.dump(dtc_best_sm, PATH_PRODUCTION_MODEL)

['../models/best_model.pkl']

In [32]:
# Update the configuration parameter.
config = update_config(
    key = "path_production_model",
    value = PATH_PRODUCTION_MODEL,
    params = config,
    path_config = PATH_CONFIG
)

Params Updated! 
Key: path_production_model 
Value: ../models/best_model.pkl

